In [15]:
import numpy as np
import soundfile as sf


def to_generate_Ar_Av(sa_freq, duration,notes):
    ratios = {
            'Sa': 1.0, 
            'Re': 9/8,      #R2
            'Ga': 6/5,      # G₂
            'Ma': 4/3,      # Ma
            'Pa': 3/2,      # Pa
            'Dha': 27/16,  # D2
            'Ni': 9/5,     # Ni2
            'S': 2.0      
        }

    sr = 22050  # sample rate
    audio = np.array([])

    for r in notes:
        f = sa_freq * ratios[r]
        t = np.linspace(0, duration, int(sr*duration), endpoint=False)
        wave = 0.5*np.sin(2*np.pi*f*t)
        audio = np.concatenate((audio, wave))
    return audio

sa_freq = 261.63
duration = 0.8
notes = ["Sa","Ga","Re","Ga","Ma","Pa","Dha","Pa","S","S","Ni","Dha","Pa","Ma","Ga","Re","Sa"]
audio = to_generate_Ar_Av(sa_freq, duration, notes)
sf.write("anandhabairavi.wav", audio, sr)
print("Audio file saved as anandhabairavi.wav")

# S G₂ R₂ G₂ M₁ P D₂ P Ṡ[a]// Ṡ N₂ D₂ P M₁ G₂ R₂ S[b]

Audio file saved as anandhabairavi.wav


Ārohanam: S G₂ R₂ G₂ M₁ P D₂ P S \
Avarohanam: S N₂ D₂ P M₁ G₂ R₂ S

In [12]:
import numpy as np
import librosa
import soundfile as sf

def predict_swaras(audio_path, sa_freq, dur):
    """
    Predicts swaras (Sa, Re, Ga, ...) from an audio file given reference Sa frequency.
    
    Args:
        audio_path (str): Path to the audio file (.wav).
        sa_freq (float): Reference Sa frequency in Hz.
    
    Returns:
        list: Sequence of swara names corresponding to each 0.8s segment.
    """

    # Swara ratios (Just Intonation, one common mapping)
    ratios = {
            'Sa': 1.0, 
            'Re': 9/8,      #R2
            'Ga': 6/5,      # G₂
            'Ma': 4/3,      # Ma
            'Pa': 3/2,      # Pa
            'Dha': 27/16,  # D2
            'Ni': 9/5,     # Ni2
            'S': 2.0      
        }

    
    # Compute swara frequencies
    swara_freqs = {s: sa_freq * r for s, r in ratios.items()}
    
    # Load audio
    y, sr = librosa.load(audio_path, sr=None)
    
    # Each note lasts 0.8s
    segment_len = int(duration * sr)
    n_segments = len(y) // segment_len
    
    swaras_predicted = []
    
    for i in range(n_segments):
        segment = y[i*segment_len : (i+1)*segment_len]
        
        # Estimate pitch (fundamental frequency)
        f0, _, _ = librosa.pyin(segment, 
                                fmin=sa_freq/2, fmax=sa_freq*3, 
                                sr=sr, frame_length=2048)
        f0 = f0[~np.isnan(f0)]
        if len(f0) == 0:
            swaras_predicted.append("Unknown")
            continue
        freq_est = np.median(f0)  # representative frequency of this note
        
        # Find closest swara
        closest_swara = min(swara_freqs.keys(), 
                            key=lambda s: abs(swara_freqs[s] - freq_est))
        
        swaras_predicted.append(closest_swara)
    
    return swaras_predicted

dur = 0.8
sa_freq = 261.63
result = predict_swaras("anandhabairavi.wav", sa_freq, dur)
print(result)

['Sa', 'Ga', 'Re', 'Ga', 'Ma', 'Pa', 'Dha', 'Pa', 'S', 'S', 'Ni', 'Dha', 'Pa', 'Ma', 'Ga', 'Re', 'Sa']


In [16]:
n2 = ["Pa", "Ma", "Ga", "Re", "Sa", 
      "Sa", "Re", "Ga", "Ma", 
      "Pa", "Dha", "Pa", "Ma", "Ga",
      "Ma", "Ga", "Re", "Sa", "Ni", 
      "Dha", "Pa", "Ma", "Ga", "Re", "Sa",
      "Sa", "Re", "Ga", "Ma", 
      "Pa", "Ma", "Ga", "Re",
      "Ga", "Re", "Sa", 
      "Ni", "Dha", "Pa"]

au = to_generate_Ar_Av(sa_freq, duration, n2)
sf.write("anandhabairavi_ch.wav", au, sr)
print("Audio file saved as anandhabairavi_ch.wav")

Audio file saved as anandhabairavi_ch.wav


In [17]:
import numpy as np
import librosa
import soundfile as sf

def predict_swaras(audio_path, sa_freq, dur):
    """
    Predicts swaras (Sa, Re, Ga, ...) from an audio file given reference Sa frequency.
    
    Args:
        audio_path (str): Path to the audio file (.wav).
        sa_freq (float): Reference Sa frequency in Hz.
    
    Returns:
        list: Sequence of swara names corresponding to each 0.8s segment.
    """

    # Swara ratios (Just Intonation, one common mapping)
    ratios = {
            'Sa': 1.0, 
            'Re': 9/8,      #R2
            'Ga': 6/5,      # G₂
            'Ma': 4/3,      # Ma
            'Pa': 3/2,      # Pa
            'Dha': 27/16,  # D2
            'Ni': 9/5,     # Ni2
            'S': 2.0      
        }

    
    # Compute swara frequencies
    swara_freqs = {s: sa_freq * r for s, r in ratios.items()}
    
    # Load audio
    y, sr = librosa.load(audio_path, sr=None)
    
    # Each note lasts 0.8s
    segment_len = int(duration * sr)
    n_segments = len(y) // segment_len
    
    swaras_predicted = []
    
    for i in range(n_segments):
        segment = y[i*segment_len : (i+1)*segment_len]
        
        # Estimate pitch (fundamental frequency)
        f0, _, _ = librosa.pyin(segment, 
                                fmin=sa_freq/2, fmax=sa_freq*3, 
                                sr=sr, frame_length=2048)
        f0 = f0[~np.isnan(f0)]
        if len(f0) == 0:
            swaras_predicted.append("Unknown")
            continue
        freq_est = np.median(f0)  # representative frequency of this note
        
        # Find closest swara
        closest_swara = min(swara_freqs.keys(), 
                            key=lambda s: abs(swara_freqs[s] - freq_est))
        
        swaras_predicted.append(closest_swara)
    
    return swaras_predicted

dur = 0.8
sa_freq = 261.63
result = predict_swaras("anandhabairavi_ch.wav", sa_freq, dur)
print(result)

['Pa', 'Ma', 'Ga', 'Re', 'Sa', 'Sa', 'Re', 'Ga', 'Ma', 'Pa', 'Dha', 'Pa', 'Ma', 'Ga', 'Ma', 'Ga', 'Re', 'Sa', 'Ni', 'Dha', 'Pa', 'Ma', 'Ga', 'Re', 'Sa', 'Sa', 'Re', 'Ga', 'Ma', 'Pa', 'Ma', 'Ga', 'Re', 'Ga', 'Re', 'Sa', 'Ni', 'Dha', 'Pa']
